In [1]:
# STL
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
import logging
# 3rd Party
import torch
from transformers import AutoTokenizer
# Local
from gatbert.data import parse_graph_tsv
from gatbert.graph_sample import GraphSample, Edge
from gatbert.constants import Stance
from gatbert.utils import batched, change_log_level

/home/ethanlmines/blue_dir/conda_envs/gatbert/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pretrained_model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, use_fast=True, add_prefix_space=True)

In [3]:
import logging
from contextlib import contextmanager
@contextmanager
def change_log_level(logger_name: str, log_level: int = logging.ERROR):
    try:
        logger = logging.getLogger(logger_name)
        old_level = logger.getEffectiveLevel()
        logger.setLevel(log_level)
        yield
    finally:
        logger.setLevel(old_level)

In [4]:
graph_gen = parse_graph_tsv('scrap.tsv')
graph_gen = map(lambda sample: sample.encode(tokenizer), graph_gen)
graph_gen = batched(graph_gen, 4)
graph_gen = map(GraphSample.collate, graph_gen)

with change_log_level("transformers.tokenization_utils_base"):
    for batch in graph_gen:
        print(*[t.shape for t in batch.values()])
        break

torch.Size([4, 344]) torch.Size([4, 197, 344]) torch.Size([4, 34871])
